In [91]:
import pandas as pd
import urllib
import json
import numpy as np
import csv

In [16]:
#sluzebny
r = urllib.request.urlopen('http://mapakriminality.cz/api/areas?level=3')
data = json.loads(r.read().decode(r.info().get_param('charset') or 'utf-8'))

arrs = {}
ind = 0
for ar in data['areas']:
    ind += 1
    arrs[ind] = ar
    
sluzebny = pd.DataFrame.from_dict(arrs, orient='index')

In [17]:
out = {}

for row in sluzebny.iterrows():
    code = row[1]['Code']
    r = urllib.request.urlopen('http://mapakriminality.cz/api/crimes?areacode=' + code + '&crimetypes=411&timefrom=1-2013&timeto=11-2015&groupby=time')
    resp = json.loads(r.read().decode(r.info().get_param('charset') or 'utf-8'))['crimes']
    
    ret = {}
    for month in resp:
        ret[month['TimePeriod']] = {i:month[i] for i in month if i!='TimePeriod'}
    out[code] = ret

In [89]:
#transpose
transout = {}
for dist in out:
    dats = []
    if dist not in transout:
        transout[dist] = {'data': {}}
    
    for month in out[dist]:
        try:
            dats.append(float(out[dist][month]['CrimeRate']))
        except:
            continue
        try:
            transout[dist]['data'][month] = float(out[dist][month]['Found'])
        except:
            continue
    transout[dist]['rate'] = np.mean(dats)

C:\Anaconda3\lib\site-packages\numpy\core\_methods.py:59: RuntimeWarning: Mean of empty slice.
  warnings.warn("Mean of empty slice.", RuntimeWarning)


In [92]:
with open('data/transposed.csv', 'w', newline='') as csvfile:
    spamwriter = csv.writer(csvfile, delimiter=';', quotechar='|', quoting=csv.QUOTE_MINIMAL)
    spamwriter.writerow(['oopID', 'rate', 'data'])
    for oop in transout:
        spamwriter.writerow([oop, transout[oop]['rate'], transout[oop]['data']])